In [11]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install scikit-learn
## note i installed pip install matplotlic and swarmpackagepy in terminal meaning in project


  Cloning https://github.com/7ossam81/SwarmPackagePy.git to c:\users\dell\appdata\local\temp\pip-req-build-a_6anjth


  Running command git clone --filter=blob:none --quiet https://github.com/7ossam81/SwarmPackagePy.git 'C:\Users\Dell\AppData\Local\Temp\pip-req-build-a_6anjth'
  remote: Repository not found.
  fatal: repository 'https://github.com/7ossam81/SwarmPackagePy.git/' not found
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/7ossam81/SwarmPackagePy.git 'C:\Users\Dell\AppData\Local\Temp\pip-req-build-a_6anjth' did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/7ossam81/SwarmPackagePy.git 'C:\Users\Dell\AppData\Local\Temp\pip-req-build-a_6anjth' did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
# Required Libraries:
# pip install numpy pandas scikit-learn

import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# --- Cuckoo Search Implementation ---
def levy_flight(Lambda):
    sigma = (math.gamma(1 + Lambda) * math.sin(math.pi * Lambda / 2) /
             (math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
    u = np.random.normal(0, sigma, size=1)
    v = np.random.normal(0, 1, size=1)
    step = u / (np.abs(v) ** (1 / Lambda))
    return step

class CuckooSearch:
    def __init__(self, obj_func, num_nests=15, dimensions=8, iterations=100, pa=0.25, alpha=0.01):
        self.obj_func = obj_func
        self.num_nests = num_nests
        self.dimensions = dimensions
        self.iterations = iterations
        self.pa = pa
        self.alpha = alpha

    def initialize_nests(self):
        return np.random.rand(self.num_nests, self.dimensions)

    def get_best_nest(self, nests, fitness):
        idx = np.argmin(fitness)
        return nests[idx].copy(), fitness[idx]

    def empty_nests(self, nests, fitness):
        K = np.random.rand(*nests.shape) > self.pa
        new_nests = nests + 0.01 * np.random.randn(*nests.shape) * K
        new_nests = np.clip(new_nests, 0, 1)
        new_fitness = np.array([self.obj_func(n) for n in new_nests])
        improved = new_fitness < fitness
        fitness[improved] = new_fitness[improved]
        nests[improved] = new_nests[improved]
        return nests, fitness

    def run(self):
        nests = self.initialize_nests()
        fitness = np.array([self.obj_func(n) for n in nests])
        best_nest, best_fitness = self.get_best_nest(nests, fitness)

        for _ in range(self.iterations):
            for i in range(self.num_nests):
                step = levy_flight(1.5)
                step = step * (nests[i] - best_nest)
                new_nest = nests[i] + self.alpha * step
                new_nest = np.clip(new_nest, 0, 1)
                new_fitness = self.obj_func(new_nest)
                if new_fitness < fitness[i]:
                    fitness[i] = new_fitness
                    nests[i] = new_nest
                if new_fitness < best_fitness:
                    best_nest = new_nest.copy()
                    best_fitness = new_fitness
            nests, fitness = self.empty_nests(nests, fitness)

        return best_nest, best_fitness

# --- Load Dataset ---
df = pd.read_csv("diabetes.csv")
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# --- Objective function for Cuckoo Search ---
def objective_function(weights):
    X_weighted = X_train * weights
    model = LogisticRegression(max_iter=200)
    model.fit(X_weighted, y_train)
    y_pred = model.predict(X_weighted)
    return 1 - accuracy_score(y_train, y_pred)

# --- Run Cuckoo Search ---
cs = CuckooSearch(
    obj_func=objective_function,
    num_nests=15,
    dimensions=X_train.shape[1],
    iterations=100,
    alpha=0.01,
    pa=0.25
)

best_weights, best_score = cs.run()

print("Best fitness (1 - accuracy):", best_score)
print("Best weights:", best_weights)

# --- Train final model ---
final_model = LogisticRegression(max_iter=200)
final_model.fit(X_train * best_weights, y_train)
y_pred_test = final_model.predict(X_test * best_weights)
test_acc = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {test_acc:.4f}")

# --- Predict diabetes on user input ---
features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
            'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

print("\nEnter patient details to predict diabetes:")

user_input = [float(input(f"{feature}: ")) for feature in features]
user_scaled = scaler.transform([user_input])
user_weighted = user_scaled * best_weights
prediction = final_model.predict(user_weighted)
print("Prediction:", "Diabetic" if prediction[0] == 1 else "Not Diabetic")


Best fitness (1 - accuracy): 0.21661237785016285
Best weights: [0.60076423 0.3623682  0.80449704 0.60215539 0.03811673 0.93484275
 0.54915823 0.07304977]
Test Accuracy: 0.7857

Enter patient details to predict diabetes:
Prediction: Diabetic


d:\diabetes_project\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
